In [22]:
# Import libraries
import cudf
import pandas as pd
from numba import cuda
import shapefile as sh
import numpy as np
import math

In [23]:
# Read in US Tiger Line Edge shapefile (Example here is King County)
shp_name = "/data/us_tiger/king_county/tl_2018_53033_edges.shp"
sf = sh.Reader(shp_name)

In [24]:
# Get field positions for road type and nodes
fields = list(zip(*sf.fields))
mtfcc_pos = fields[0].index('MTFCC') - 1
roadflg_pos = fields[0].index('ROADFLG') - 1
nodef_pos = fields[0].index('TNIDF') - 1
nodet_pos = fields[0].index('TNIDT') - 1

In [25]:
# Parse shape file into list of edges and their constituent vectors
vectors = []
edges = []
# Read nodes into a dictionary to de-duplicate
nodes = {}
node_num = 0
mtfcc = []
# Iterate through shapefile shape objects
for i,geo in enumerate(sf.shapes()):
    # Get the shape attributes
    attr = sf.record(i)
    # Check that the edge is a road
    if attr[roadflg_pos] == 'Y':
        # Test if mtfcc (edge classification) is in list, if not add it
        if attr[mtfcc_pos] not in mtfcc:
            mtfcc += [attr[mtfcc_pos]]
        # Test if from node exists
        if attr[nodef_pos] not in nodes:
            # If not, add it to the dictionary with coordinates of first point of vector
            nodes[attr[nodef_pos]] = [node_num,geo.points[0][0],geo.points[0][1]]
            node_num += 1
        # Test if to node exists
        if attr[nodet_pos] not in nodes:
            # If not, add it to the dictionary with coordinates of last point of vector
            nodes[attr[nodet_pos]] = [node_num,geo.points[-1][0],geo.points[-1][1]]
            node_num += 1
        # Add edge to table 
        edges += [[i,nodes[attr[nodef_pos]][0],nodes[attr[nodet_pos]][0],attr[mtfcc_pos]]]
        # Add edge to table with from and too nodes reversed
        edges += [[i,nodes[attr[nodet_pos]][0],nodes[attr[nodef_pos]][0],attr[mtfcc_pos]]]
        # Unravel and serialise the vector points into a list
        for p in geo.points:
            vectors += [[i,p[0],p[1]]]
# Sort mtfcc
mtfcc.sort()

In [26]:
# Zip the vector points and copy to cudf DataFrame
vz = list(zip(*vectors))
vector_df = cudf.DataFrame()
vector_df['id'] = vz[0]
vector_df['lon'] = vz[1]
vector_df['lat'] = vz[2]

In [27]:
# Zip the node points and copy to cudf DataFrame
nz = list(zip(*sorted(nodes.values())))
node_df = cudf.DataFrame()
node_df['id'] = nz[0]
node_df['lon'] = nz[1]
node_df['lat'] = nz[2]

In [28]:
# Zip the edges and copy to cudf DataFrame
ez = list(zip(*edges))
edge_df = cudf.DataFrame()
edge_df['id'] = ez[0]
edge_df['src'] = ez[1]
edge_df['dst'] = ez[2]
edge_df['mtfcc'] = [mtfcc.index(m) for m in ez[3]]

In [29]:
# Check results on vector Dataframe
print(vector_df)

            id         lon        lat
0            1 -121.781978  47.408331
1            1 -121.781920  47.408344
2            2 -121.781920  47.408344
3            2 -121.780622  47.408632
4            3 -122.188432  47.318418
5            3 -122.188419  47.318221
6            3 -122.188425  47.318168
7            3 -122.188512  47.318034
8            4 -122.038309  47.244633
9            4 -122.038323  47.244663
10           4 -122.038358  47.244685
11           4 -122.038403  47.244691
12           4 -122.038447  47.244681
13           4 -122.038473  47.244665
14           4 -122.038488  47.244638
15           4 -122.038486  47.244604
16           4 -122.038468  47.244582
17           4 -122.038427  47.244564
18           4 -122.038389  47.244561
19           4 -122.038351  47.244570
20           4 -122.038319  47.244593
21           4 -122.038308  47.244618
22           4 -122.038309  47.244633
23           5 -121.635090  47.233126
24           5 -121.635018  47.233141
25          

In [30]:
# Check results on node Dataframe
print(node_df)

            id         lon        lat
0            0 -121.781978  47.408331
1            1 -121.781920  47.408344
2            2 -121.780622  47.408632
3            3 -122.188432  47.318418
4            4 -122.188512  47.318034
5            5 -122.038309  47.244633
6            6 -121.635090  47.233126
7            7 -121.630742  47.232446
8            8 -121.354531  47.274795
9            9 -121.356002  47.276650
10          10 -121.837180  47.185363
11          11 -121.836133  47.184873
12          12 -122.121523  47.250319
13          13 -122.120557  47.250328
14          14 -122.020366  47.177900
15          15 -122.021445  47.178649
16          16 -122.019788  47.176851
17          17 -122.020842  47.177496
18          18 -122.199217  47.313017
19          19 -122.199808  47.313996
20          20 -121.758851  47.223611
21          21 -121.758743  47.223744
22          22 -122.103135  47.224471
23          23 -122.102282  47.224134
24          24 -122.006546  47.319999
25          

In [31]:
# Check results on edge DataFrame
print(edge_df)

            id     src     dst  mtfcc
0            1       0       1      2
1            1       1       0      2
2            2       1       2      2
3            2       2       1      2
4            3       3       4      2
5            3       4       3      2
6            4       5       5      2
7            4       5       5      2
8            5       6       7      2
9            5       7       6      2
10           6       8       9      2
11           6       9       8      2
12           7      10      11      2
13           7      11      10      2
14           8      12      13      2
15           8      13      12      2
16           9      14      15      2
17           9      15      14      2
18          10      16      17      2
19          10      17      16      2
20          11      18      19      2
21          11      19      18      2
22          12      20      21      2
23          12      21      20      2
24          13      22      23      2
25          

In [32]:
# Define chunk size and parameters
threads_per_block = 128
trunk_size = 10240
data_length = vector_df['id'].count()

In [33]:
# Haversine function for lengths
@cuda.jit(device=True)
def haversine_distance(lon_1, lat_1, lon_2, lat_2):
    lon_1 = lon_1 * math.pi / 180.0 
    lon_2 = lon_2 * math.pi / 180.0 
    lat_1 = lat_1 * math.pi / 180.0 
    lat_2 = lat_2 * math.pi / 180.0
    d_lon = lon_2 - lon_1 
    d_lat = lat_2 - lat_1 
    a = math.sin(d_lat/2)**2 + math.cos(lat_1) * math.cos(lat_2) * math.sin(d_lon/2)**2
    return 2 * math.asin(math.sqrt(a)) * 6371000.0

In [34]:
# Window function to calculate distance between adjacent points in feet
def adjacent_distance(id, lon, lat, dist):
    for i in range(cuda.threadIdx.x, id.size, cuda.blockDim.x):
        # If the first point of vector return zero length
        if i == 0:
            dist[i] = 0.0
        # Or if the vector changes (1st point of new vector)
        elif id[i] != id[i-1]:
            dist[i] = 0.0
        else:
            dist[i] = haversine_distance(lon[i],lat[i],lon[i-1],lat[i-1]) * 3.280839895013123

In [35]:
%%time
# Apply the window function
vector_df = vector_df.apply_chunks(adjacent_distance,
                     incols=['id','lon','lat'],
                     outcols=dict(dist=np.float64),
                     kwargs=dict(),
                     chunks=list(range(0, data_length,
                                       trunk_size))+ [data_length],
                     tpb=threads_per_block)

CPU times: user 453 ms, sys: 4.15 ms, total: 458 ms
Wall time: 457 ms


In [36]:
# Check the result
print(vector_df)

            id         lon        lat        dist
0            1 -121.781978  47.408331    0.000000
1            1 -121.781920  47.408344   15.084758
2            2 -121.781920  47.408344    0.000000
3            2 -121.780622  47.408632  337.250971
4            3 -122.188432  47.318418    0.000000
5            3 -122.188419  47.318221   71.939992
6            3 -122.188425  47.318168   19.391934
7            3 -122.188512  47.318034   53.410635
8            4 -122.038309  47.244633    0.000000
9            4 -122.038323  47.244663   11.480474
10           4 -122.038358  47.244685   11.813160
11           4 -122.038403  47.244691   11.357620
12           4 -122.038447  47.244681   11.491489
13           4 -122.038473  47.244665    8.690997
14           4 -122.038488  47.244638   10.527198
15           4 -122.038486  47.244604   12.413520
16           4 -122.038468  47.244582    9.180824
17           4 -122.038427  47.244564   12.092395
18           4 -122.038389  47.244561    9.474526


In [37]:
%%time
# Group by and sum the distances for each edge
distance_df = vector_df[['id','dist']].groupby(['id']).sum()
distance_df['id'] = distance_df.index

CPU times: user 21.4 ms, sys: 4.21 ms, total: 25.7 ms
Wall time: 24.1 ms


In [38]:
%%time
# Merge the distances into the edge DataFrame to produce graph
edge_df = edge_df.merge(distance_df,on=['id'], how='left').sort_values('id')

CPU times: user 31.6 ms, sys: 4.28 ms, total: 35.9 ms
Wall time: 34.3 ms


In [39]:
# Check Result
print(edge_df)

            id     src     dst  mtfcc         dist
18464        1       0       1      2    15.084758
18465        1       1       0      2    15.084758
18466        2       1       2      2   337.250971
18467        2       2       1      2   337.250971
18468        3       3       4      2   144.742561
18469        3       4       3      2   144.742561
18470        4       5       5      2   145.027620
18471        4       5       5      2   145.027620
18472        5       6       7      2  1116.975442
18473        5       7       6      2  1116.975442
18474        6       8       9      2   937.954515
18475        6       9       8      2   937.954515
18476        7      10      11      2   315.341735
18477        7      11      10      2   315.341735
18478        8      12      13      2   239.236642
18479        8      13      12      2   239.236642
18480        9      14      15      2   382.502849
18481        9      15      14      2   382.502849
18482       10      16      17 

In [40]:
# Save road graph to csv
graph_file = shp_name[:-4] + "_graph.csv"
edge_df.to_pandas().to_csv(graph_file, index=False)

In [41]:
# Save road modes to csv
node_file = shp_name[:-4] + "_nodes.csv"
node_df.to_pandas().to_csv(node_file, index=False)

In [42]:
# print mtfcc table (used to filter on road type)
print([(i,m) for i,m in enumerate(mtfcc)])

[(0, 'S1100'), (1, 'S1200'), (2, 'S1400'), (3, 'S1500'), (4, 'S1630'), (5, 'S1640'), (6, 'S1710'), (7, 'S1720'), (8, 'S1730'), (9, 'S1740'), (10, 'S1750'), (11, 'S1780'), (12, 'S1820')]
